In [187]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [188]:
import matplotlib.pyplot as plt
import polars as pl
from datetime import datetime, time

from ts.supplier import TradeTick
from ts.supplier import TickSupplier

from ts.supplier import Bar
from ts.supplier import BarSupplier
from ts.supplier import BarAggregation
from ts.supplier import BarFeatureSupplier, BarFeatures
from ts.supplier import RollingFeaturesSupplier, SupplierType
from ts.supplier import Function, BaseSupplier
from ts.supplier import MultiplexSupplier

filepath = "/Users/jacobgora/projects/data/continuous_futures/CME-HO.parquet"
tick_supplier = TickSupplier(instrument="CME-HO")
tick_supplier.from_parquet(filepath)

In [189]:
suppliers = [
    BarFeatureSupplier(
        supplier=BarSupplier(
            supplier=tick_supplier,
            bar_aggregation=BarAggregation.TIME_MINUTES,
            size=5
        )
    ),
    BarFeatureSupplier(
        supplier=BarSupplier(
            supplier=tick_supplier,
            bar_aggregation=BarAggregation.TIME_MINUTES,
            size=1
        )
    )
]

multiplex_supplier = MultiplexSupplier(suppliers=suppliers)

In [190]:
rolling_feat_supplier = RollingFeaturesSupplier(
    supplier=multiplex_supplier,
    functions=[Function.Z_SCORE],
    type_attributes=[
        # OFI / Tape
        BarFeatures.OFI_NORMALIZED,
        BarFeatures.OFI,

        BarFeatures.ASK_SIZE,
        BarFeatures.BID_SIZE,

        BarFeatures.ASK_SIZE_TIMEDELTA,
        BarFeatures.BID_SIZE_TIMEDELTA,

        BarFeatures.RETURN_TIMEDELTA,

        BarFeatures.RETURN_BID_SIZE,
        BarFeatures.RETURN_ASK_SIZE,

        # Volume
        BarFeatures.CUMULATIVE_VOLUME_DELTA,
        BarFeatures.VOLUME_DELTA,
        BarFeatures.VOLUME,

        # Volatility
        BarFeatures.POS_REALIZED_VARIANCE,
        BarFeatures.NEG_REALIZED_VARIANCE,

        #
        BarFeatures.INTERNAL_BAR_STRENGTH
    ]
)

In [197]:
rolling_feat_supplier.data[
    [
        e for e in rolling_feat_supplier.data.columns
        if "rolling_features" in e
    ]
].tail()

rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ofi_normalized-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ofi_normalized-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ofi-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ofi-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ask_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ask_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-bid_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-bid_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-return_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-return_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-volume_delta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-volume_delta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-volume-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-volume-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-internal_bar_strength-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-internal_bar_strength-z_score-10
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.659223,0.680917,1.037562,17.078371,-0.432018,-0.845872,NaN,NaN,0.041977,-0.382497,-1.037562,-17.078371,0.236711,17.938541,NaN,-1.873314
0.303857,0.659366,1.523175,17.01613,-0.432018,-0.794414,NaN,NaN,-0.02426,-0.575144,-1.523175,-17.01613,1.147112,17.927575,NaN,-1.720809
-1.453718,0.318571,-1.444478,16.690439,-0.376443,-0.697195,NaN,NaN,0.190516,-0.626803,1.444478,-16.690439,0.23825,17.902931,NaN,-1.525668
0.149386,-0.326637,8.409404,16.148576,-0.460838,-1.58917,NaN,NaN,0.096814,-0.684028,-8.409404,-16.148576,12.198141,17.862311,NaN,-1.372188
-0.473929,-1.522993,4.253971,13.932494,-0.594195,-1.600055,NaN,NaN,0.173925,-0.731367,-4.253971,-13.932494,10.06066,25.767978,NaN,-1.788854
